In [ ]:
%python
%pip install faker

In [ ]:
%python
dbutils.library.restartPython() 

In [ ]:
%python
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from faker import Faker
import random


spark = SparkSession.builder.getOrCreate()
fake = Faker("pt_BR")
Faker.seed(42)
random.seed(42)

In [ ]:
%python
from pyspark.sql.types import StructType, StructField, StringType
import random
from faker import Faker

fake = Faker('pt_BR')  # Gera dados brasileiros

qtde_clientes = 300

# Carregar a tabela bancos como DataFrame
df_bancos = spark.table("corebank_catalog.bronze_corebank.bancos")

# Extrair os ids dos bancos
bancos_ids = [r.id_banco for r in df_bancos.select("id_banco").collect()]

clientes_outros_data = []
for i in range(1, qtde_clientes + 1):
    sigla_estado = fake.estado_sigla()
    nome_estado = fake.estado_nome()
    cidade = fake.city()
    id_banco = random.choice(bancos_ids)
    nome = fake.name()
    # CPF: se não tiver o método, gere um número fictício
    try:
        cpf = fake.cpf()
    except AttributeError:
        cpf = ''.join([str(random.randint(0, 9)) for _ in range(11)])
    telefone = fake.phone_number()
    email = fake.email()
    endereco = fake.address().replace("\n", ", ")
    id_cliente = f"C{i:06d}"
    # Data de nascimento
    try:
        data_nascimento = fake.date_of_birth().isoformat()
    except AttributeError:
        data_nascimento = fake.date_of_birth().isoformat()
    uuid = fake.uuid4()
    chave_pix = random.choice([cpf, telefone, email])  # Pix key is randomly one of these

    clientes_outros_data.append((
        nome,
        cpf,
        telefone,
        email,
        endereco,
        id_cliente,
        str(id_banco),  # Garante que é string
        sigla_estado,
        cidade,
        data_nascimento,
        uuid,
        chave_pix,
    ))

schema_clientes_outros = StructType([
    StructField("nome_completo", StringType(), True),
    StructField("cpf_cnpj", StringType(), True),
    StructField("telefone", StringType(), True),
    StructField("email", StringType(), True),
    StructField("endereco", StringType(), True),
    StructField("id_cliente", StringType(), True),
    StructField("id_banco", StringType(), True),
    StructField("estado", StringType(), True),
    StructField("cidade", StringType(), True),
    StructField("data_nascimento", StringType(), True),
    StructField("uuid", StringType(), True),
    StructField("chave_pix", StringType(), True),
])

df_clientes_outros = spark.createDataFrame(
    clientes_outros_data,
    schema_clientes_outros
)
df_clientes_outros.createOrReplaceTempView("clientes_outros")
df_clientes_outros.write \
    .option("mergeSchema", "true") \
    .mode("overwrite") \
    .saveAsTable("corebank_catalog.bronze_corebank.clientes_outros")
display(df_clientes_outros)
